# Genomic Data - DemoHumanOrWorm.ipynb

This is one of the genomic datasets taken from [here](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks).
The classification task is evaluated using the _SeqRep_ package.

You can [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MIR-MU/seqrep/blob/main/examples/genomic_data/DemoHumanOrWorm.ipynb)
or
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/MIR-MU/seqrep/main?labpath=examples%2Fgenomic_data%2FDemoHumanOrWorm.ipynb).

## Install _SeqRep_ Package

In [1]:
!pip install seqrep

     |████████████████████████████████| 115 kB 14.4 MB/s 
     |████████████████████████████████| 303 kB 49.5 MB/s 
     |████████████████████████████████| 15.3 MB 426 kB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=eaeddced6fc4a1b55769449da0a6e256080717d0ed4d3ab1374c3ed24b4b2866
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
  Created wheel for ta: filename=ta-0.9.0-py3-none-any.whl size=28908 sha256=e14d8389dd0a029541e3d446b91c84051b62b91fafcadfcf69aa6e2a703eb990
  Stored in directory: /root/.cache/pip/wheels/72/78/64/cc1c01506a1010a9845e9bd7c69333730f7174661228ea4f98
Successfully built pandas-ta ta
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully 

## Import Needed Packages

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPClassifier

!pip install icecream
from icecream import ic

from seqrep import *
from seqrep.feature_engineering import *
from seqrep.labeling import *
from seqrep.splitting import *
from seqrep.scaling import *
from seqrep.feature_reduction import *
from seqrep.evaluation import *
from seqrep.pipeline_evaluation import *

## Load or Download Data

In [4]:
!pip install genomic-benchmarks

     |████████████████████████████████| 2.3 MB 9.3 MB/s 
     |████████████████████████████████| 596 kB 56.8 MB/s 
     |████████████████████████████████| 271 kB 55.9 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
  Created wheel for genomic-benchmarks: filename=genomic_benchmarks-0.0.6-py3-none-any.whl size=17526 sha256=5251a6d08b38c0d1d31db12508d283aa350aae4e27d4583d919ddd511d56e404
  Stored in directory: /root/.cache/pip/wheels/a7/0f/e4/f723fa5070923dc495b63e02600faf64bf76dbdf422ebdeb66
Successfully built genomic-benchmarks
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
from genomic_benchmarks.data_check import list_datasets

list_datasets()

['demo_coding_vs_intergenomic_seqs',
 'demo_human_or_worm',
 'human_enhancers_cohn',
 'human_enhancers_ensembl',
 'human_nontata_promoters',
 'demo_mouse_enhancers']

In [6]:
["".join([w.capitalize() for w in dat.split("_")]) for dat in list_datasets()]

['DemoCodingVsIntergenomicSeqs',
 'DemoHumanOrWorm',
 'HumanEnhancersCohn',
 'HumanEnhancersEnsembl',
 'HumanNontataPromoters',
 'DemoMouseEnhancers']

In [7]:
# demo_human_or_worm
from genomic_benchmarks.dataset_getters.pytorch_datasets import (
    DemoHumanOrWorm,
)

X_train = DemoHumanOrWorm(split="train", version=0)
X_test = DemoHumanOrWorm(split="test", version=0)


y_train = pd.Series([y for _, y in X_train])
X_train = pd.DataFrame([x for x, _ in X_train], columns=["genom"])
y_test = pd.Series([y for _, y in X_test])
X_test = pd.DataFrame([x for x, _ in X_test], columns=["genom"])

Unzipping...Done.


In [8]:
## Random shuffling
# idx = np.random.permutation(len(y_train))
# X_train = X_train.iloc[idx, :].reset_index(drop=True)
# y_train = y_train.iloc[idx].reset_index(drop=True)

# idx = np.random.permutation(len(y_test))
# X_test = X_test.iloc[idx, :].reset_index(drop=True)
# y_test = y_test.iloc[idx].reset_index(drop=True)

X_train.join(pd.DataFrame(y_train, columns=["label"]))

,genom,label
0,CTAAAAATACAAAAATTAGCTGGGTGTGGTGGCGCGCGCCTGTAAT...,0
1,CTGGTGATGCTGGAAGCATTGGATGCCCTGTAAGGACATGATTTTG...,0
2,ATTAAAAGCATACTTGTTCAAATTTGGTATAAATAGGACATATTAC...,0
3,GGAGGCCAAGGCGGGTGGATCACCTGAGGTCGGGCGTTCAAGACCA...,0
4,TATAAGACCTAAAGGCAGCAACTAGCTAATATCTGTCCAGTGTTAT...,0
...,...,...
74995,CGAAGTTTGGTTCTCGGATTGTGTGCTGGCACTTTCCTGCCAAATG...,1
74996,AGACACCCTGAGAGTCGATTTGTCTCATTTTTCGTCGATAAATGTA...,1
74997,CGTATCTCTGGTTGCCAGTTTATTTCTACGATGAGCCATTTCAATT...,1
74998,TTTCGTTCCATGCATCAATGTCTAATCCAGCCTTCATAGAGTTTCT...,1


## Run Pipeline Evaluation

In [9]:
# This DataFrame collects the results of various runs for comparison.

# Uncomment following line if you want to clear the DataFrame with the results.
# del results_for_comparison

try:
    results_for_comparison
except NameError:
    print("Create new empty DataFrame.")
    results_for_comparison = pd.DataFrame()
else:
    print("DataFrame already exist!")

Create new empty DataFrame.


In [11]:
class SubstringsExtractor(FeatureExtractor):
    def __init__(
        self,
        substrings: List,
        columns_to_apply: Union[str, List[str]] = None,
        return_original_columns: bool = False,
        normalize: bool = True,
        verbose: bool = True,
        inplace: bool = False,
    ):
        self.substrings = substrings
        self.columns_to_apply = columns_to_apply
        self.return_original_columns = return_original_columns
        self.normalize = normalize
        self.verbose = verbose
        self.inplace = inplace

    def fit(self, X, y=None):
        if self.verbose:
            print(f"\tNumber of substrings BEFORE fit: {len(self.substrings)}")
        if not self.columns_to_apply:
            self.columns_to_apply = X.columns
        if isinstance(self.columns_to_apply, str):
            columns_to_apply = [self.columns_to_apply]

        tmp = ""
        for c in self.columns_to_apply:
            tmp += "@".join(X[c])
        self.substrings = [
            s
            for s in tqdm(
                self.substrings, leave=False, desc="Fitting SubstringsExtractor"
            )
            if s in tmp
        ]
        del tmp

        if self.verbose:
            print(f"\tNumber of substrings AFTER fit:  {len(self.substrings)}")
        return self

    def transform(self, X):
        if not self.inplace:
            X = X.copy()
        for column in tqdm(
            self.columns_to_apply,
            leave=False,
            desc="Transforming SubstringsExtractor - columns",
        ):
            col_pref = column + "_" if len(self.columns_to_apply) > 1 else ""
            for substr in tqdm(
                self.substrings,
                leave=False,
                desc="Transforming SubstringsExtractor - substrings",
            ):
                X.loc[:, f"{col_pref}count-{substr}"] = (
                    X[column].str.count(substr) / X[column].str.len()
                    if self.normalize
                    else 1
                )

        if self.return_original_columns:
            return X
        return X.drop(columns=self.columns_to_apply)

In [17]:
from itertools import product

perms = [
    "".join(p)
    for i in range(1, 7)
    for p in product("ACTG", repeat=i)
    if len(p) < 5 or len(set(p)) > 2
]
ic(len(perms))
# ic(perms[:10])

ic| len(perms): 4900


4900

In [13]:
%%capture --no-stdout --no-display
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

run_identification = f"{len(perms)} substrings"

# 1. step - define your pipeline
pipe = Pipeline(
    [
        (
            "fext_substr",
            SubstringsExtractor(
                substrings=perms,
            ),
        ),
        ("scale_u", UniversalScaler(scaler=MinMaxScaler())),
    ]
)

# 2. step - define your workflow
pipe_eval = PipelineEvaluator(
    pipeline=pipe,
    model=MLPClassifier(
        hidden_layer_sizes=(128, 32, 8),
        batch_size=32,
    ),
    evaluator=SequentialEvaluator(
        [
            ClassificationEvaluator(),
            UniversalEvaluator(metrics=[f1_score]),
        ]
    ),
)
# 3. step
pipe_eval.X_train = X_train.copy()
pipe_eval.y_train = y_train.copy()
pipe_eval.X_test = X_test.copy()
pipe_eval.y_test = y_test.copy()

result = pipe_eval.run()

results_for_comparison = results_for_comparison.append(
    pd.Series(result, name=run_identification),
)

07:37:50.506 Fitting pipeline
	Number of substrings BEFORE fit: 4900


Fitting SubstringsExtractor:   0%|          | 0/4900 [00:00<?, ?it/s]

	Number of substrings AFTER fit:  4900


Transforming SubstringsExtractor - columns:   0%|          | 0/1 [00:00<?, ?it/s]

Transforming SubstringsExtractor - substrings:   0%|          | 0/4900 [00:00<?, ?it/s]

07:51:51.558 Applying pipeline transformations


Transforming SubstringsExtractor - columns:   0%|          | 0/1 [00:00<?, ?it/s]

Transforming SubstringsExtractor - substrings:   0%|          | 0/4900 [00:00<?, ?it/s]

07:56:40.792 	Original shape:		(75000, 4900); 
		shape after removing NaNs: (74998, 4900).
07:56:42.184 	Original shape:		(25000, 4900); 
		shape after removing NaNs: (25000, 4900).
07:56:42.185 Fitting model
12:20:39.190 Predicting
12:20:58.483 Evaluating predictions
[[11808   692]
 [  502 11998]] 
 95.224 % accuracy
 94.54688731284476 % precision of 1 classes
 95.98400000000001 % recall of 1 classes

              precision    recall  f1-score   support

           0       0.96      0.94      0.95     12500
           1       0.95      0.96      0.95     12500

    accuracy                           0.95     25000
   macro avg       0.95      0.95      0.95     25000
weighted avg       0.95      0.95      0.95     25000

f1_score:
	0.9526002381897579


In [14]:
results_for_comparison

,accuracy_score,precision_score,recall_score,confusion_matrix,f1_score
4900 substrings,0.95224,0.945469,0.95984,"[[11808, 692], [502, 11998]]",0.9526


| Dataset                          |   Accuracy |   F1 score |   |
|:---------------------------------|-----------:|-----------:|-----------:|
| demo_human_or_worm |    95.224 |     95.26 | 4900 substrings	 |
| demo_human_or_worm |    0 |     0 |  |
| demo_human_or_worm |    0 |     0 |   |